Introductory example to using GPry
----------------------------------

This notebook will guide you through the basic steps for sampling a likelihood with GPry

### Step 1: Setting up a likelihood function

Let's set up a simple 2d Gaussian likelihood as an example:

$$y(x) \sim \mathcal{N}(x|\mu,\Sigma)$$

with 
$$\mu=\begin{pmatrix}3 \\ 2\end{pmatrix},\ \Sigma=\begin{pmatrix}0.5 & 0.4 \\ 0.4 & 1.5\end{pmatrix}$$

**Note that we need to pass GPry the log-Likelihood!**

In [ ]:
import os
import numpy as np
from scipy.stats import multivariate_normal

mean = [3, 2]
cov = [[0.5, 0.4], [0.4, 1.5]]
rv = multivariate_normal(mean, cov)

def logLkl(x_1, x_2):
    return rv.logpdf(np.array([x_1, x_2]).T)

and some bounds which we set to $[-10, 10]$ for both paramters

In [ ]:
bounds = [[-10, 10], [-10, 10]]

### Step 2: Creating the Runner object

The `run.Runner` manages model specification and the active sampling loop of GPry up to convergence, as well as allows for some post-processing and tests.

To initialise it, we pass it the log-likelihood function as first argument, and the prior bounds via the ``bounds`` keyword (more complicated prior specifications can be used by defining and passing as first argument a `Cobaya` model).

Optionally, we can also also pass a path to save checkpoints via the ``checkpoint`` argument. 

If passed, in order to prevent loss of data, you **must** decide a checkpoint policy (either ``"resume"`` or ``"overwrite"``). 

- If set to ``"resume"`` the runner object will try to load the checkpoint and resume the active sampling loop from there
- If set to ``"overwrite"`` it will start from scratch and overwrite checkpoint files which already exist.


In [ ]:
from gpry.run import Runner
checkpoint = "output/simple"
runner = Runner(logLkl, bounds, checkpoint=checkpoint, load_checkpoint="overwrite")

### Step 3: Running the active learning loop

Since all training parameters are chosen automatically, all we have to do is to call the
`run` function of the runner object:


In [ ]:
runner.run()

This will run the active sampling loop until convergence is reached. 

It also saves the checkpoint files after every iteration of the bayesian optimization loop and creates progress plots which are saved in `[checkpoint]/images/` (`./images/` if checkpoint is None).

Once converged, you can access the surrogate model and use it as a function for any purpose. 

Note that internally GPry models the **log-posterior**, not the log-likelihood and that you need to hand GPry a single (nsamples, ndim) with the locations where you want to evaluate the surrogate.

Nevertheless we can call either `runner.logp` or `runner.logL`

Let us compare GPry and the likelihood in the location (1, 2): 

In [ ]:
point = (1, 2)
print(f"Log-lkl at (1,2): {logLkl(*point)}")
print(f"surrogate at (1,2): {runner.logL(point)[0]}")

Both evaluations should produce similar numbers.

Congratulations, now you have a surrogate of your posterior distribution!!!

----

Now let's see how you make corner plots and get marginalised quantities.

### Step 4: Monte Carlo samples on the surrogate model

The ``Runner`` object can also run an MC sampler on the GP in order to extract marginalised quantities. To do that, we use the ``generate_mc_sample`` method of the ``Runner``.

By default, GPry would already have run an MC sampler at the end of the main loop, for diagnosis purposes. You can get the result using the ``last_mc_samples`` method, which returns a dictionary containing the samples' parameter values, weight (``None`` if all samples carry equal weight), and values for the surrogate log-posterior, true log- prior, and surrogate log-likelihood (i.e., the surrogate log-posterior minus the analytic log-prior):

In [ ]:
mc_samples_dict = runner.last_mc_samples(as_pandas=True)
print(mc_samples_dict)

Samples are also stored by default in the same folder as the checkpoint, inside a ``chains`` sub folder. The order of the columns in that file are ``weight log-posterior param_1 param_2 ...``.

Subsequent calls to the ``generate_mc_sample`` method can be used to re-generate MC samples from the surrogate posterior, e.g. if a finer representation is needed.

In [ ]:
runner.generate_mc_sample(
    # Example args for denser samples
    # sampler={"nested": {"nlive": "25d", "num_repeats": "10d"}}
)

### Plotting the results

Now that we have MC samples you can process and plot them the same way that you would do with any other MC samples.

The easiest way to get a corner plot though is to call the `plot_mc` method of the ``Runner`` object which will generate a corner plot.

It includes the training set unless passed ``add_training=False``.

In [ ]:
runner.plot_mc()

### Bonus: Getting some extra insights

You can do further plots about the progress of the active-learning loop using:

In [ ]:
runner.plot_progress()

This results in the following: 
- a histogram of the distribution time spent at different parts of the code (`timing.png`)
- the distribution of the training samples (`trace.png`)
- A plot showing the value(s) of all convergence criteria as function of the number of posterior evaluations (`convergence.png`)

The plots are saved in `[checkpoint]/images/` (`./images/` if checkpoint is None).

### Bonus Bonus: Validation

**This part is optional and only relevant for validating the contours that GPry produces. In a realistic scenario you would obviously not run a full MCMC on the likelihood**

To compare our contours to the true Gaussian we draw 10000 samples from it, and set them as *fiducial samples* in the ``Runner``:

In [ ]:
truth_samples = rv.rvs(size=10000)
runner.set_fiducial_MC(truth_samples)

runner.plot_mc()